# ДЗ Парсер сайта

http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/

#### Библиотеки

In [1]:
#!pip3 install requests bs4 pandas
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

#### Получение и преобразование html страницы списка квартир и страницы самой квартиры

In [2]:
table_content = requests.get('http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/')
table_content

<Response [200]>

In [3]:
table_content = BeautifulSoup(table_content.text, 'lxml')
#table_content # разобрали страницу на понятный, более менее упорядоченный html код

In [4]:
flat_page = requests.get('http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/prodam-kvartiru-pravoberezhnyy-23240323.html')
flat_page = BeautifulSoup(flat_page.text, 'lxml')
#flat_page #посмотрим на код страницы квартиры, чтобы понять, как нам извлекать данные

### Извлечем параметры квартиры с каждой из страниц

Перед извлечением параметров было проведено ручное исследование страниц квартир и страниц со спискоми квартир. Было выявлено, что информация на странице квартиры наиболее полная, а также хранится в более удобном формате. Следовательно парсить данные по квартирам со страницы со списком квартир имеет смысл только для получения айди объявления. Этот айди затем будет использоваться в качестве индекса в нашей таблице с данными.
<img src="../datasets/flat_page.png" width="500"> <img src="../datasets/table_content.png" width="550">

#### 1) Парсим идентификаторы квартир

In [5]:
# Для начала найдем сколько страниц нам придется парсить
div = table_content.find('div', 'pager__pages')
div

<div class="pager__pages"><span class="pager__page_state_current">1</span><a class="pager__page" href="/realty/prodazha-kvartir/?p=2">2</a><a class="pager__page" href="/realty/prodazha-kvartir/?p=3">3</a><a class="pager__page" href="/realty/prodazha-kvartir/?p=4">4</a><a class="pager__page" href="/realty/prodazha-kvartir/?p=5">5</a><a class="pager__page common-link-visited" href="/realty/prodazha-kvartir/?p=6">...</a><a class="pager__page" href="/realty/prodazha-kvartir/?p=17">17</a></div>

In [6]:
numeric_list = [int(i.get_text()) for i in div.find_all('a') if i.get_text().isdigit()]
print(numeric_list)
page_count = max(numeric_list) # Лучше было сделать так: page_count = int(list(div)[-1].get_text())
page_count # Значение можно было получить взяв последний элемент списка, но мало ли как расположит автор сайта нумерацию страниц :)

[2, 3, 4, 5, 17]


17

In [7]:
list_pages = range(1, page_count + 1) # Прямо во время выполнения задания страниц стало 21, вместо 20)
list_pages

range(1, 18)

При просмотре кода страницы с объявлениями видно, что айди квартиры находится в первом и во втором столбцах таблицы. Мы будем брать данные из первого столбца.
<img src="../datasets/flat_id.png">

In [8]:
# Находим на странице таблицу, отсеиваем ненужные строки, получаем список из айдишников квартир
table = table_content.find('table', 'table')
#table

In [9]:
# Нас инетересует содержание столбеца notebook-column
tr_list = [td for td in table.find_all('tr', recursive=False) if td.find('td', 'notebook-column')]
#tr_list[0] # 30 штук

In [10]:
id_list = [i.find('input', {'name': 'cs_note_tg'})['value'] for i in tr_list]
link_list = [i.find_all('a')[0]['href'] for i in tr_list]
#list(zip(id_list, link_list)) # тоже 30 штук, осталось поставить все "на поток"

In [11]:
#  Вынесем получение списка айдишек в отдельную функцию

SITE = "http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/"

def get_id_list(num_page):
    if num_page == 1:
        page = requests.get(SITE)
    else:
        page = requests.get(SITE + f'?p={num_page}')
    page = BeautifulSoup(page.text, 'lxml')
    tr_list = [td for td in page.find('table', 'table').find_all('tr', recursive=False) if td.find('td', 'notebook-column')]
    id_list = [i.find('input', {'name': 'cs_note_tg'})['value'] for i in tr_list]
    link_list = [i.find_all('a')[0]['href'] for i in tr_list]

    return id_list, link_list

In [12]:
%%time

full_id_list, full_link_list = [], []
for num_page in list_pages:
    id_list, link_list = get_id_list(num_page) 
    full_id_list.extend(id_list)
    full_link_list.extend(link_list)

#full_id_list, , full_link_list
len(full_id_list), len(full_link_list)

CPU times: user 1.54 s, sys: 161 ms, total: 1.71 s
Wall time: 3min 32s


(504, 504)

In [13]:
len(set(full_id_list)), len(set(full_link_list))

(504, 504)

#### 2) Создаем колонки

Данные будут собираться по 11 параметрам: Ссылка, Цена, Район, Адрес, Этаж, Высота потолка, Планировка, Площадь, Состояние квартиры, Кол-во комнат, Описание.

Из объявлений квартир видно, что данные где-то заполнены не полностью, соответственно в конечной таблице у нас будут пропуски. Так как на каждой странице квартиры таблица с данными разного размера, то ориентироваться будем по названию строк, а затем вставлять их в нужную колонку в нашем датафрейме.

<img src='../datasets/flat_1.png' width=400> <img src='../datasets/flat_2.png' width=400>

In [14]:
# Создаем колонки
columns = ['Ссылка', 'Цена', 'Район', 'Адрес', 'Этаж',
           'Высота потолка', 'Планировка', 'Площадь', 
           'Состояние квартиры', 'Кол-во комнат', 'Описание']
columns

['Ссылка',
 'Цена',
 'Район',
 'Адрес',
 'Этаж',
 'Высота потолка',
 'Планировка',
 'Площадь',
 'Состояние квартиры',
 'Кол-во комнат',
 'Описание']

#### 3) Парсим данные со страницы и через словарь вставляем их в датафрем

Оказалось, что просто идентификатора объявления недостаточно, чтобы перейти на страницу этого объявления...

Так как:
- http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/prodam-kvartiru-23257029.html
- http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/prodam-kvartiru-leninskiy-23257030.html
- http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/prodam-kvartiru-ordzhonikidzevskiy-23182211.html
- http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/prodam-kvartiru-pravoberezhnyy-23188503.html

То есть если квартира в описании имеет название района, то в адресной строке добавляется название этого района, если же район не указан, то окончание строки просто "prodam-kvartiru-айди объявления". Следовательно, чтобы перейти на страницу объявления мы вынуждены вместе с айдишниками парсить район.

Тут я хотел было расстроиться, затем просто вписать вручную транслитерацию районов (их оказывается всего 4), но заметил, что все гораздо проще. Каждая строка таблицы (как же это было очевидно...) имеет внутри себя ссылку на само объявление. Поэтому вместе с айдишками, можно сразу собирать ссылки, которые потом использовать для перехода и сбора данных.

In [15]:
# Возвращаемся на пункт "Парсим идентификаторы квартир"
...
# Теперь все отлично, можно создать датафрейм

In [16]:
df = pd.DataFrame(index=full_id_list, columns=columns)
df['Ссылка'] = full_link_list
df.head()

,Ссылка,Цена,Район,Адрес,Этаж,Высота потолка,Планировка,Площадь,Состояние квартиры,Кол-во комнат,Описание
23417266,/realty/prodazha-kvartir/prodam-kvartiru-pravo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23408391,/realty/prodazha-kvartir/prodam-kvartiru-23408...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23408389,/realty/prodazha-kvartir/prodam-kvartiru-23408...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19673514,/realty/prodazha-kvartir/prodam-kvartiru-ordzh...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23163976,/realty/prodazha-kvartir/prodam-kvartiru-lenin...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/prodam-kvartiru-pravoberezhnyy-23188503.html
flat_page =  requests.get('http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/prodam-kvartiru-pravoberezhnyy-23188503.html')
flat_page = BeautifulSoup(flat_page.text, 'lxml')
#flat_page

In [18]:
div = flat_page.find('div', 'adv-main-data')
#div

In [19]:
tr_list = div.find_all('tr')
#tr_list

In [20]:
# field-title – ключ к словарю, field – его значение, поиск в словаре происходит за O(1), так что все ок
# Подумал сначала сразу получить разделенные данные, но получить их в грязном виде,
# а потом почистить пандасом оказалось проще, чем сразу собрать чистые как надо
# плюс это позволит не заморачиваться над тем, на какую позицию придется ставить найденный элемент

key_list = [key.get_text() for key in div.find_all('td', 'field-title')]
value_list = [value.get_text().replace('\xa0', '') for value in div.find_all('td', 'field')]

data_dict = dict(zip(key_list, value_list))
data_dict['Описание'] = div.find('td', 'note').get_text()
data_dict

# P.S. не понимал почему у одних строк на этой странице класс even, а другие без класса, а потом как дошло...

{'Цена': '4050000р. (66069р./м2)Подать заявку на ипотеку',
 'Район': 'Правобережный',
 'Адрес': 'Суворова, 139',
 'Этаж': '2/5',
 'Высота потолка': '2,5 м.',
 'Планировка': 'хрущевка',
 'Площадь': 'общая 61,3 м2, жилая 46 м2, кухни 6 м2',
 'Состояние квартиры': 'Среднее',
 'Кол-во комнат': 'Трехкомнатная',
 'Этажность дома': '5',
 'Описание': '🏠 Продается уютная 3-комнатная квартира 61,3 м² в отличном состоянии\n📍 Адрес: ул. Суворова, 139, 2 этаж 5-этажного блочного дома\n🔍 О квартире:\n•📐 Площадь: 61.3 м²\n•🏢 Этаж: 2/5\n•🧭 Ориентация окон: Восток-Запад → светлая, весь день солнце\n•🪟 Окна: Пластиковые стеклопакеты\n•🧊 Балкон: Застекленный с выносом\n•⚡ Электрика: Медная проводка\n•💧 Коммуникации: Трубы пластик, установлены водомеры\n•🚻 Санузел: Раздельный в кафеле\n•🧱 Отделка: Полы — стяжка, в коридоре — кафель. Стены ровные. Потолки: натяжной и двухуровневый из гипсокартона.\n🛋️ Остается вся мебель и техника:\n•🍽️ Кухня: кухонный гарнитур, обеденный стол\n•🚪 Прихожая: мебельная стенк

In [21]:
BASE_SITE = 'http://magnitogorsk-citystar.ru'

def get_data(link):
    flat_page = BeautifulSoup(requests.get('http://magnitogorsk-citystar.ru' + link).text, 'lxml')
    div = flat_page.find('div', 'adv-main-data')
    key_list = [key.get_text() for key in div.find_all('td', 'field-title')]
    value_list = [value.get_text().replace('\xa0', '') for value in div.find_all('td', 'field')]
    data_dict = dict(zip(key_list, value_list))
    if div.find('td', 'note'):
        data_dict['Описание'] = div.find('td', 'note').get_text()
    return data_dict

In [22]:
%%time

for ind in full_id_list:
    df.loc[str(ind), 'Цена':] = get_data(df.loc[str(ind), 'Ссылка'])

df.head()

CPU times: user 18.2 s, sys: 1.21 s, total: 19.4 s
Wall time: 7min 44s


,Ссылка,Цена,Район,Адрес,Этаж,Высота потолка,Планировка,Площадь,Состояние квартиры,Кол-во комнат,Описание
23417266,/realty/prodazha-kvartir/prodam-kvartiru-pravo...,4890000р. (78871р./м2)Подать заявку на ипотеку,Правобережный,"Им. газеты ""Правда"", 62/2",2/5,NaN,NaN,"общая 62 м2, жилая 45 м2, кухни 8 м2",Хорошее,Трехкомнатная,💥 Квартира в очень хорошем состоянии. 👍\n\n✅ В...
23408391,/realty/prodazha-kvartir/prodam-kvartiru-23408...,3800000р. (82969р./м2)Подать заявку на ипотеку,NaN,"Карла Маркса, 112",2/5,NaN,NaN,"общая 45,8 м2, жилая 31 м2, кухни 7 м2",NaN,Двухкомнатная,id:46213. \nПродается уютная двухкомнатная ква...
23408389,/realty/prodazha-kvartir/prodam-kvartiru-23408...,4950000р. (85052р./м2)Подать заявку на ипотеку,NaN,"Октябрьская, 32/1",2/5,NaN,NaN,"общая 58,2 м2, жилая 40,4 м2, кухни 6,3 м2",NaN,Трехкомнатная,id:46096. \nПродаётся 3-комнатная квартира в Л...
19673514,/realty/prodazha-kvartir/prodam-kvartiru-ordzh...,6590000р. (63365р./м2)Подать заявку на ипотеку,Орджоникидзевский,"Советская, 213/1",14/14,NaN,нестандартная,"общая 104 м2, жилая 66,3 м2, кухни 15,8 м2",Хорошее,Четырехкомнатная,💥 КВАРТИРА ОТЛИЧНОЙ НЕСТАНДАРТНОЙ ПЛАНИРОВКИ С...
23163976,/realty/prodazha-kvartir/prodam-kvartiru-lenin...,3790000р. (67679р./м2)Подать заявку на ипотеку,Ленинский,"Московская, 26/1",3/4,NaN,NaN,"общая 56 м2, жилая 37 м2, кухни 6 м2",Хорошее,Трехкомнатная,⭐ КВАРТИРА ТРЕХКОМНАТНАЯ ПОЛУСМЕЖНАЯ. ⭐\n\n✅ С...


#### 4) Приводим собранные данные в порядок

In [24]:
df[['Общая площадь', 'Жилая площадь', 'Площадь кухни']] = df['Площадь'].str.split(' ', expand=True).loc[:, [1, 4, 7]]
df.drop('Площадь', axis=1)
#df.head()

,Ссылка,Цена,Район,Адрес,Этаж,Высота потолка,Планировка,Состояние квартиры,Кол-во комнат,Описание,Общая площадь,Жилая площадь,Площадь кухни
23417266,/realty/prodazha-kvartir/prodam-kvartiru-pravo...,4890000р. (78871р./м2)Подать заявку на ипотеку,Правобережный,"Им. газеты ""Правда"", 62/2",2/5,NaN,NaN,Хорошее,Трехкомнатная,💥 Квартира в очень хорошем состоянии. 👍\n\n✅ В...,62,45,8
23408391,/realty/prodazha-kvartir/prodam-kvartiru-23408...,3800000р. (82969р./м2)Подать заявку на ипотеку,NaN,"Карла Маркса, 112",2/5,NaN,NaN,NaN,Двухкомнатная,id:46213. \nПродается уютная двухкомнатная ква...,"45,8",31,7
23408389,/realty/prodazha-kvartir/prodam-kvartiru-23408...,4950000р. (85052р./м2)Подать заявку на ипотеку,NaN,"Октябрьская, 32/1",2/5,NaN,NaN,NaN,Трехкомнатная,id:46096. \nПродаётся 3-комнатная квартира в Л...,"58,2","40,4","6,3"
19673514,/realty/prodazha-kvartir/prodam-kvartiru-ordzh...,6590000р. (63365р./м2)Подать заявку на ипотеку,Орджоникидзевский,"Советская, 213/1",14/14,NaN,нестандартная,Хорошее,Четырехкомнатная,💥 КВАРТИРА ОТЛИЧНОЙ НЕСТАНДАРТНОЙ ПЛАНИРОВКИ С...,104,"66,3","15,8"
23163976,/realty/prodazha-kvartir/prodam-kvartiru-lenin...,3790000р. (67679р./м2)Подать заявку на ипотеку,Ленинский,"Московская, 26/1",3/4,NaN,NaN,Хорошее,Трехкомнатная,⭐ КВАРТИРА ТРЕХКОМНАТНАЯ ПОЛУСМЕЖНАЯ. ⭐\n\n✅ С...,56,37,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16262155,/realty/prodazha-kvartir/prodam-kvartiru-ordzh...,5500000р. (84615р./м2)Подать заявку на ипотеку,Орджоникидзевский,"Жукова, 19/1",1/10,NaN,NaN,NaN,Двухкомнатная,"<p>В продаже уютная 2к квартира, в одном из лу...",65,43,12
15406262,/realty/prodazha-kvartir/prodam-kvartiru-ordzh...,8000000р. (76190р./м2)Подать заявку на ипотеку,Орджоникидзевский,"Карла Маркса, 185",4/9,NaN,NaN,NaN,Четырехкомнатная,Предлагаю Вашему вниманию четырёхкомнатную ква...,105,83,10
16022577,/realty/prodazha-kvartir/prodam-kvartiru-lenin...,5560000р. (76374р./м2)Подать заявку на ипотеку,Ленинский,"Строителей, 57",6/6,NaN,NaN,NaN,Трехкомнатная,<p>Для ценителей Ленинского района в продаже у...,"72,8",44,8
15761115,/realty/prodazha-kvartir/prodam-kvartiru-15761...,1950000р. (108333р./м2)Подать заявку на ипотеку,NaN,"Курортная, 19",2/2,NaN,NaN,NaN,Однокомнатная,"В эксклюзивной продаже апартаменты в <b>ЖК""КАР...",18,15,0


In [25]:
df[['Этаж', 'Этажность дома']] = df['Этаж'].str.split('/', expand=True)
#df.head()

In [26]:
df['Цена'] = df['Цена'][:].str.split('р.', expand=True).loc[:, 0]
#df.head()

In [27]:
df['Описание'] = df['Описание'].str.replace('^id:\\d+\\.\\s*', '', regex=True)
df.head()

,Ссылка,Цена,Район,Адрес,Этаж,Высота потолка,Планировка,Площадь,Состояние квартиры,Кол-во комнат,Описание,Общая площадь,Жилая площадь,Площадь кухни,Этажность дома
23417266,/realty/prodazha-kvartir/prodam-kvartiru-pravo...,4890000,Правобережный,"Им. газеты ""Правда"", 62/2",2,NaN,NaN,"общая 62 м2, жилая 45 м2, кухни 8 м2",Хорошее,Трехкомнатная,💥 Квартира в очень хорошем состоянии. 👍\n\n✅ В...,62,45,8,5
23408391,/realty/prodazha-kvartir/prodam-kvartiru-23408...,3800000,NaN,"Карла Маркса, 112",2,NaN,NaN,"общая 45,8 м2, жилая 31 м2, кухни 7 м2",NaN,Двухкомнатная,Продается уютная двухкомнатная квартира на ком...,"45,8",31,7,5
23408389,/realty/prodazha-kvartir/prodam-kvartiru-23408...,4950000,NaN,"Октябрьская, 32/1",2,NaN,NaN,"общая 58,2 м2, жилая 40,4 м2, кухни 6,3 м2",NaN,Трехкомнатная,Продаётся 3-комнатная квартира в Ленинском рай...,"58,2","40,4","6,3",5
19673514,/realty/prodazha-kvartir/prodam-kvartiru-ordzh...,6590000,Орджоникидзевский,"Советская, 213/1",14,NaN,нестандартная,"общая 104 м2, жилая 66,3 м2, кухни 15,8 м2",Хорошее,Четырехкомнатная,💥 КВАРТИРА ОТЛИЧНОЙ НЕСТАНДАРТНОЙ ПЛАНИРОВКИ С...,104,"66,3","15,8",14
23163976,/realty/prodazha-kvartir/prodam-kvartiru-lenin...,3790000,Ленинский,"Московская, 26/1",3,NaN,NaN,"общая 56 м2, жилая 37 м2, кухни 6 м2",Хорошее,Трехкомнатная,⭐ КВАРТИРА ТРЕХКОМНАТНАЯ ПОЛУСМЕЖНАЯ. ⭐\n\n✅ С...,56,37,6,4


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 504 entries, 23417266 to 14046477
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Ссылка              504 non-null    object
 1   Цена                503 non-null    object
 2   Район               185 non-null    object
 3   Адрес               499 non-null    object
 4   Этаж                504 non-null    object
 5   Высота потолка      28 non-null     object
 6   Планировка          56 non-null     object
 7   Площадь             504 non-null    object
 8   Состояние квартиры  62 non-null     object
 9   Кол-во комнат       495 non-null    object
 10  Описание            502 non-null    object
 11  Общая площадь       504 non-null    object
 12  Жилая площадь       504 non-null    object
 13  Площадь кухни       504 non-null    object
 14  Этажность дома      504 non-null    object
dtypes: object(15)
memory usage: 79.2+ KB


#### 5) Сохраням файл в удобный формат

In [29]:
df.to_csv('../datasets/flat_magnitogorsk.csv')

### Заключение

Сайт делал какой-то извращенец, задание интересное, понравилось, ≈6 часов точно ушло. 

- Парсер первый — айди и ссылки на страницы квартир работает около 2 минут.
- Парсер второй – основные данные по квартирам работает около 5 минут.

Осталось только преобразовать в необходимые типы данных.